In [ ]:
import numpy as np
import pandas as pd
from pylab import plot,show,hist,bar
import gc
%config IPCompleter.greedy=True


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split

data = pd.read_csv("../input/train.tsv",sep="\t")
#data = pd.read_csv("../data/test.csv",sep="\t")
print(len(data))
#data = data.sample#(frac=0.5)
print(len(data))

In [ ]:
train_data = pd.read_csv("../input/mercari-price-suggestion-challenge/train.tsv",sep="\t")
train_data['log_price'] = np.log(train_data.price+1)
train_data=train_data[train_data.price>2]
test_data = pd.read_csv("../input/mercari-price-suggestion-challenge/test.tsv",sep="\t")

n_trains = train_data.shape[0]
n_tests = test_data.shape[0]

data = pd.concat([train_data,test_data])

# Overall description 

### Observations
1. Few category names missing
2. 42% of brand_name in missing 1
3. category_name , brand_name , item_description have some missing values

### observation
1. Price is highly skewed
2. Log price looks good

# Item_condition_id


In [ ]:
icidf=pd.get_dummies(data.item_condition_id)
data['ici_1']=icidf[icidf.columns[0]]
data['ici_2']=icidf[icidf.columns[1]]
data['ici_3']=icidf[icidf.columns[2]]
data['ici_4']=icidf[icidf.columns[3]]
data['ici_5']=icidf[icidf.columns[4]]


#data['median_item_condition_id']=data.item_condition_id.map({1:18.0,2:17.0,3:16.0,4:15.0,5:19.0})
#print data.item_condition_id

it_id_rv=data[['log_price','item_condition_id']].groupby(['item_condition_id']).describe()
it_id_rv_dict = dict(zip(it_id_rv['log_price']['mean'].index,it_id_rv['log_price']['mean']))
data['mean_it_id']=list(map(lambda x:it_id_rv_dict[x],data.item_condition_id))
del it_id_rv
del it_id_rv_dict

### observation 
1. 5 condition is really low , but has the highest median price ( maybe means good condition )
2. 4 count is low , even its median price is low ( maybe means really bad condition )

###  observation 
1. from log price plot we see , that almost for all categories the distribution is same except that it is somewhat different for 5th

# Category_name  





In [ ]:
data['Cat1']=data.category_name.str.split('/').str.get(0)
data['Cat2']=data.category_name.str.split('/').str.get(1)
data['Cat3']=data.category_name.str.split('/').str.get(2)
data.Cat1= data.Cat1.fillna('NotPresent')
data.Cat2= data.Cat2.fillna('NotPresent')
data.Cat3= data.Cat3.fillna('NotPresent')

In [ ]:
cat1_rv=data[['log_price','Cat1']].groupby(['Cat1']).describe()
cat1_rv_dict = dict(zip(cat1_rv['log_price']['mean'].index,cat1_rv['log_price']['mean']))
data['mean_Cat1']=list(map(lambda x:cat1_rv_dict[x],data.Cat1))
#print data.mean_Cat1
del cat1_rv
del cat1_rv_dict

In [ ]:
cat2_rv=data[['log_price','Cat2']].groupby(['Cat2']).describe()
cat2_rv_dict = dict(zip(cat2_rv['log_price']['mean'].index,cat2_rv['log_price']['mean']))
data['mean_Cat2']=list(map(lambda x:cat2_rv_dict[x],data.Cat2))
del cat2_rv
del cat2_rv_dict

In [ ]:
cat3_rv=data[['log_price','Cat3']].groupby(['Cat3']).describe()
cat3_rv_dict = dict(zip(cat3_rv['log_price']['mean'].index,cat3_rv['log_price']['mean']))
data['mean_Cat3']=list(map(lambda x:cat3_rv_dict[x],data.Cat3))
del cat3_rv
del cat3_rv_dict

In [ ]:
data=data.drop(['Cat1','Cat2','Cat3','category_name'],axis=1)
gc.collect()

### Cat1

len(data.Cat1.unique())

plot(xrange(10),data[['price','Cat1']].groupby(['Cat1']).median(),'-x')
show()

data[['price','Cat1']].groupby(['Cat1']).describe()#.sort_values(by='count', ascending=False)

hist(np.log(data[data['Cat1']=='Men'].price+1))
show()

.

##  shipping


In [ ]:
dummydf=pd.get_dummies(data.shipping)
data['shipping_0']=dummydf[dummydf.columns[0]]
data['shipping_1']=dummydf[dummydf.columns[1]]

##  name

In [ ]:
data['len_name']=list(map(lambda x : len(x),data.name))

`## brand_name

In [ ]:
data.brand_name= data.brand_name.fillna('NotPresent')

In [ ]:
brand_name_rv=data[['brand_name','log_price']].groupby(['brand_name']).describe()
brand_name_rv_dict = dict(zip(brand_name_rv['log_price']['mean'].index,brand_name_rv['log_price']['mean']))
data['mean_brand_name']=list(map(lambda x:brand_name_rv_dict[x],data.brand_name))
del brand_name_rv
del brand_name_rv_dict

##  item_description 


In [ ]:
data.item_description=data.item_description.fillna('No description yet')

In [ ]:
len_desc = list(map(lambda x:len(x),data.item_description))


i=12
print data[data.price>1500].price.iloc[i]
print data[data.price>1500].brand_name.iloc[i]
print data[data.price>1500].item_description.iloc[i]
print data[data.price>1500].name.iloc[i]

# Pairwise


aa = data[['log_price','brand_name','Cat1','Cat2','Cat3','item_condition_id']].groupby(['brand_name','item_condition_id','Cat1','Cat2','Cat3']).mean().sort_values(['log_price'],ascending=False)['log_price'].to_dict()

data['letssee']=list(map(lambda x:aa[(x.brand_name,x.Cat1,x.Cat2,x.Cat3,x.item_condition_id)],data))

map(lambda y:y[1][1],data.iloc[:10].iterrows())

data.iteritems()[0]


##  Prediction 

data.Cat1= data.Cat1.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat1)
print len(le.classes_)
data['Cat1Tf'] = le.transform(data.Cat1) 


data.Cat2= data.Cat2.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat2)
print len(le.classes_)
data['Cat2Tf'] = le.transform(data.Cat2) 


data.Cat3= data.Cat3.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat3)
print len(le.classes_)
data['Cat3Tf'] = le.transform(data.Cat3) 


data.brand_name=data.brand_name.fillna('OTHERS')
le=LabelEncoder()
le.fit(data.brand_name)
print len(le.classes_)
data['brand_nameTf']= le.transform(data.brand_name) 


data['log_price'] = np.log(data.price+1)


In [ ]:
data.columnsumns

In [ ]:
data1 = data.drop(['brand_name', 'item_condition_id', 'item_description',
       'name', 'price', 'shipping'],axis=1)
print(data1.columns)

del data
gc.collect()

In [ ]:
trn_columns = ['ici_1', 'ici_2',
       'ici_3', 'ici_4', 'ici_5', 'mean_Cat1', 'mean_Cat2', 'mean_Cat3',
       'shipping_0', 'shipping_1', 'len_name', 'mean_brand_name']

targ = ['log_price']


In [ ]:
train_data=data1[:n_trains]
test_data=data1[n_trains:]
issub="Y"
if issub=="N":
    train_data,test_data = train_test_split(train_data,test_size=0.05)
    
train_data,val_data = train_test_split(train_data,test_size=0.05)
print(train_data.columns)

In [ ]:
del data1
gc.collect()

### LinearRegression

lr=LinearRegression()
lr.fit(train_data[trn_columns],train_data[targ])

preds = lr.predict(test_data[trn_columns])
print(len(preds))

np.sqrt(np.mean(np.square(preds-test_data[targ])))

### Ridge

from sklearn.linear_model import Ridge
rg=Ridge()
rg.fit(train[trn_columns],train[targ])

preds = rg.predict(test[trn_columns])
print(len(preds))

np.sqrt(np.mean(np.square(preds-test[targ])))

### SVR

svr = LinearSVR()
svr.fit(train[trn_columns],train[targ])

preds = svr.predict(test[trn_columns])
print len(preds)

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

svrr = SVR()
svrr.fit(train[trn_columns],train[targ])

preds = svrr.predict(test[trn_columns])
print len(preds)

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### RandomForest

gbr=GradientBoostingRegressor()
gbr.fit(train_data[trn_columns],train_data[targ])

preds = gbr.predict(test_data[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test_data[targ]['log_price'])
print(np.sqrt(np.mean(np.square(df.preds-df.price))))


import gc
gc.collect()
print np.sqrt(np.mean(np.square(preds-test[targ])))

# Light Gradient Boosting 

In [ ]:
print("Training...")
import lightgbm as lgb
print('Start training...')
# train
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=400,
                        learning_rate=0.2,
                        n_estimators=30)
gbm.fit(train_data[trn_columns],train_data.log_price,
        eval_set=[(val_data[trn_columns],val_data.log_price)],
        eval_metric='rmse',
        early_stopping_rounds=5)

In [ ]:
print("predicting...")  
#del train
#del data
gc.collect()
preds= gbm.predict(test_data[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print(np.sqrt(np.mean(np.square(df.preds-df.price))))

In [ ]:
sub=pd.DataFrame()
sub['test_id']=test_data.test_id.astype('int64')
sub['price']=np.exp(preds)-1
#print(sub.test_id)
sub.to_csv("sample_submission.csv",index=False)
print("Completed!!")


### Adaboost

abr=AdaBoostRegressor()
abr.fit(train[trn_columns],train[targ])

preds = abr.predict(test[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

###  SGD

sgd = SGDRegressor( n_iter=100)
sgd.fit(train[trn_columns],train[targ])

preds = sgd.predict(test[trn_columns])
print preds

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### MLP

mlp = MLPRegressor()
mlp.fit(train[trn_columns],train[targ])

preds = mlp.predict(test[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

import wordbatch
from wordbatch.models import FTRL
from wordbatch.extractors import WordBag
wb= wordbatch.WordBatch(extractor=(WordBag, {"hash_ngrams":2, "hash_ngrams_weights":[0.5, -1.0], "hash_size":2**23, "norm":'l2', "tf":'log', "idf":50.0}))
clf= FTRL(alpha=1.0, beta=1.0, L1=0.00001, L2=1.0, D=2 ** 25, iters=1)


print(train_data1.columns)

clf.fit(wb.transform(train_data1.item_description),train_data1.log_price)

preds= clf.predict(wb.transform(val_data))

print(len(preds))

print(preds)